# Importing Needed Liberaries

In [1]:
!pip install transformers datasets scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "L

In [18]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertTokenizer, BertForSequenceClassification, get_scheduler
from torch.optim import AdamW
from tqdm import tqdm
import kagglehub
import datasets
from datasets import load_dataset
import shutil
from google.colab import files

In [19]:
import transformers
print(transformers.__version__)

4.51.3


# Loading the dataset

In [5]:
kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")
path = os.path.join("/kaggle", "input", "imdb-dataset-of-50k-movie-reviews", "IMDB Dataset.csv")
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [6]:
df = pd.read_csv(path, encoding='latin-1')

In [7]:
len(df)

50000

In [8]:
print(df.columns)

Index(['review', 'sentiment'], dtype='object')


In [9]:
print(df['sentiment'].unique())

['positive' 'negative']


In [10]:
df['label'] = df['sentiment'].map({'positive': 1, 'negative': 0})

In [11]:
print(df.columns)

Index(['review', 'sentiment', 'label'], dtype='object')


In [12]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['review'].values, df['label'].values, test_size=0.2, random_state=42)

In [13]:
len(train_texts)

40000

In [14]:
len(test_texts)

10000

# Tockenization and Bert Model

In [15]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=512)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [16]:
class IMDbDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.encodings['input_ids'][idx]),
            'attention_mask': torch.tensor(self.encodings['attention_mask'][idx]),
            'labels': torch.tensor(self.labels[idx])
        }

train_dataset = IMDbDataset(train_encodings, train_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

In [3]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs.")
    model = torch.nn.DataParallel(model)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

# Training the Model on the training data

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 40
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

In [ ]:
model.train()
for epoch in range(5):
    print(f"Epoch {epoch+1}")
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss = loss.mean()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())

Epoch 1


Epoch 1: 100%|██████████| 1000/1000 [08:42<00:00,  1.91it/s, loss=0.189]


Epoch 2


Epoch 2: 100%|██████████| 1000/1000 [08:46<00:00,  1.90it/s, loss=0.0191]


Epoch 3


Epoch 3: 100%|██████████| 1000/1000 [08:45<00:00,  1.90it/s, loss=0.0184] 


Epoch 4


Epoch 4: 100%|██████████| 1000/1000 [08:45<00:00,  1.90it/s, loss=0.00129]


Epoch 5


Epoch 5: 100%|██████████| 1000/1000 [08:45<00:00,  1.90it/s, loss=0.000758]


In [ ]:
model.train()
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}")
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss = loss.mean()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())

Epoch 1


Epoch 1: 100%|██████████| 1000/1000 [08:37<00:00,  1.93it/s, loss=0.153]


Epoch 2


Epoch 2: 100%|██████████| 1000/1000 [08:40<00:00,  1.92it/s, loss=0.236] 


Epoch 3


...

Epoch 38: 100%|██████████| 1000/1000 [08:38<00:00,  1.93it/s, loss=2.38e-7]


Epoch 39


Epoch 39: 100%|██████████| 1000/1000 [08:39<00:00,  1.93it/s, loss=1.34e-7]


Epoch 40


Epoch 40:  43%|████▎     | 433/1000 [03:44<04:54,  1.92it/s, loss=1.19e-7]

# Evaluating the Model on the testing data

In [ ]:
model.eval()
preds, true_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        preds.extend(predictions.cpu().numpy())
        true_labels.extend(batch['labels'].cpu().numpy())

print(classification_report(true_labels, preds))

              precision    recall  f1-score   support

           0       0.93      0.92      0.92       999
           1       0.92      0.93      0.93      1001

    accuracy                           0.93      2000
   macro avg       0.93      0.92      0.92      2000
weighted avg       0.93      0.93      0.92      2000



# Saving the Model Files to use locally

In [ ]:
model.save_pretrained("./bert_model")
tokenizer.save_pretrained("./bert_model")
print("Model saved to 'bert_model'")

In [ ]:
shutil.make_archive('bert_model', 'zip', 'bert_model')
files.download('bert_model.zip')

In [ ]:
model.module.save_pretrained("/kaggle/working/bert_model")
tokenizer.save_pretrained("/kaggle/working/bert_model")
print("Model saved to '/kaggle/working/bert_model'")

shutil.make_archive("/kaggle/working/bert_model", 'zip', "/kaggle/working/bert_model")
print("Zipped model saved to '/kaggle/working/bert_model.zip'")

Model saved to '/kaggle/working/bert_model'
Zipped model saved to '/kaggle/working/bert_model.zip'
